In [15]:
%run 01_hash.ipynb

In [46]:
import json
from copy import deepcopy

Print objects in a nice way.

In [47]:
def stringfy(o):
    s = []
    for k,v in o.__dict__.items():
        p = f'{k}:'.ljust(15)
        if hasattr(v,'messageHash'): s.append(p+ph(v.messageHash.hex()))
        elif k == 'time'           : s.append(p+time.ctime(v))
        elif str(v)[:2]=='0x'      : s.append(p+ph(v))
        elif type(v)   ==float     : s.append(p+str(round(v,8))+' eth')
        else                       : s.append(p+str(v))
    return '\n'.join(s)

## Transaction

A transaction (abbr. tx) is used for transfering `value` from one address `fr` to another `to`.

The `fee` goes to the miner who included the tx in a block. More on mining in [03_block.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/04_mine.ipynb). The higher it is, the more likely it is included in a block.

The `nonce` is the number of transactions sent from a given address. It's used to avoid replay attacks. For a more detailed explanation, see [here](https://kb.myetherwallet.com/en/transactions/what-is-nonce/). 

It's `hash` is a unique fingerprint. Every time something in the tx changes, it gets recalculated.

In [48]:
class TX(Hashable): 
    def __init__(self, fr, to, value, fee, nonce): 
        self.fr, self.to = fr, to
        self.value       = float(value)
        self.fee         = fee
        self.nonce       = nonce
        self.time        = time.time()
        self.signed      = False
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        if prop not in ['sig','signed','hash']: 
            super().__setattr__('hash', sha(self.__dict__))
        if prop == 'sig': self.signed = True
    
    def json(self): return json.dumps({**self.__dict__}, indent=4)
    def smry(self): return f'{pmh(self.fr)} -> {pmh(self.to)} {self.value} eth'
    def __str__(self): return stringfy(self)

Create random txs.

In [49]:
tx1 = TX(rh(), rh(), 12, 0.2,  0)
tx2 = TX(rh(), rh(), 6,  0.15, 0); tx1.smry()

'📷 0x916f -> 🕔 0xeecf 12.0 eth'

### TX JSON

Every tx has a json representation that we will use for our API later.

In [56]:
tx1_json = tx1.json(); print(tx1_json)

{
    "fr": "0x916f7c0afbf1f232e92253e4e657e5926ae72b6e9b0f77cb4d794629d607c4a0",
    "hash": "0x67be1bfb64f22bf4774866a1c201139aa2e90a7e97e72a954c0c09fefe0e7692",
    "to": "0xeecf6255c7f40e98247a7322624fac6de2737fb15f0ed4bf8b6cce2aeb358ab2",
    "value": 12.0,
    "fee": 0.2,
    "nonce": 0,
    "time": 1617465845.810473,
    "signed": false
}


We can load this JSON string `d` and create a tx object.

In [57]:
def load_tx(d):
    d = json.loads(d)
    tx = TX(d['fr'],d['to'],d['value'],d['fee'],d['nonce'])
    for k,v in d.items(): setattr(tx, k, v)
    tx.hash = d['hash']
    return tx

In [58]:
tx1_from_json = load_tx(tx1_json)
assert tx1 == tx1_from_json

#### Print several txs in a nice way.

In [59]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])
print(txs2str([tx1,tx2]))

fr:            📷 0x916f7c0afb...4a0
hash:          📍 0x67be1bfb64...692
to:            🕔 0xeecf6255c7...ab2
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Sat Apr  3 18:04:05 2021
signed:        False

fr:            💺 0x5413e8dca7...b4a
hash:          📎 0x682a881478...bfb
to:            💳 0x4d8cbce4de...7ff
value:         6.0 eth
fee:           0.15 eth
nonce:         0
time:          Sat Apr  3 18:04:05 2021
signed:        False



#### Changing Tx
Every change in the object is reflected by its hash. Compare the tx hash below with the hash of unchanged tx above. The tx hash has changed. This is how we can make sure that nobody changes the tx without us knowing.

In [60]:
tx2_false_value       = deepcopy(tx2)
tx2_false_value.value = 120
print(tx2_false_value)

fr:            💺 0x5413e8dca7...b4a
hash:          🔔 0xae9b084310...b3d
to:            💳 0x4d8cbce4de...7ff
value:         120
fee:           0.15 eth
nonce:         0
time:          Sat Apr  3 18:04:05 2021
signed:        False


Transactions can be determined unequal by simply comparing the hashes as implemented in `__eq__`.

In [61]:
assert tx2 != tx2_false_value

#### Signing Tx
An account (implemented in [02_account.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/02_account.ipynb)) has the ability to sign a tx. Here we simply mock a signature with a random hash. The signature `sig` is saved as an attribute.

In [63]:
tx1_signed = deepcopy(tx1)
tx1.sig    = rh(); print(tx1)

fr:            📷 0x916f7c0afb...4a0
hash:          📍 0x67be1bfb64...692
to:            🕔 0xeecf6255c7...ab2
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Sat Apr  3 18:04:05 2021
signed:        True
sig:           🔻 0xd54e776d71...d3b


Signing the tx should not change its hash. Therefore it is equal to the unsigned tx.

In [64]:
assert tx1 == tx1_signed